<a href="https://colab.research.google.com/github/Sudhamsh1/Data_Analytics/blob/main/Pyspark_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install Pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=0f566ee79b169c62c443aa534346c56ed23c97dd11323542a5da605190d21de1
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built Pyspark


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark=SparkSession.builder.appName("Practice").getOrCreate()



In [8]:
data=[[1,5],[1,5],[1,5],[2,6],[1,5]]
df=spark.createDataFrame(data,["a","b"])
df.show()

+---+---+
|  a|  b|
+---+---+
|  1|  5|
|  1|  5|
|  1|  5|
|  2|  6|
|  1|  5|
+---+---+



In [17]:
df_1=df.filter(df.a==1)
df_1=df_1.agg(sum(df_1.b))
df_1.show()

df_2=df.withColumn("c",when(df.a==1,20).otherwise(df.b))

df_2.show()

+------+
|sum(b)|
+------+
|    20|
+------+

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  5| 20|
|  1|  5| 20|
|  1|  5| 20|
|  2|  6|  6|
|  1|  5| 20|
+---+---+---+



In [20]:
1#)Create a Dataframe from existing one in such a way that all the columns should have single valued attribute
#1NF

data=(['Alice',['Tennis','batminton']],['bob',['Tennis','Cricket']],['Julie',['Cricket','Carroms']])

df1=spark.createDataFrame(data,['name','hobbies'])

df1.select(df1.name,explode(df1.hobbies).alias('hobbies')).show()

+-----+---------+
| name|  hobbies|
+-----+---------+
|Alice|   Tennis|
|Alice|batminton|
|  bob|   Tennis|
|  bob|  Cricket|
|Julie|  Cricket|
|Julie|  Carroms|
+-----+---------+



In [37]:
data=[('Goa', '', 'AP'), ('', 'AP', None), (None, '', 'Blr')]
schema="City1 string, City2 string, City3 string"
df2=spark.createDataFrame(data, schema)
df2.show()
df3=df2.withColumn('city1',when(df2.City1=="",None).otherwise(df2.City1))\
       .withColumn('city2',when(df2.City2=="",None).otherwise(df2.City2))\
       .withColumn('city3',when(df2.City3=="",None).otherwise(df2.City3))

df3.withColumn("Result",coalesce(df3.city1,df3.city2,df3.city3)).show()






+-----+-----+-----+
|City1|City2|City3|
+-----+-----+-----+
|  Goa|     |   AP|
|     |   AP| NULL|
| NULL|     |  Blr|
+-----+-----+-----+

+-----+-----+-----+------+
|city1|city2|city3|Result|
+-----+-----+-----+------+
|  Goa| NULL|   AP|   Goa|
| NULL|   AP| NULL|    AP|
| NULL| NULL|  Blr|   Blr|
+-----+-----+-----+------+



In [40]:
data=[('Goa', '', 'AP'), ('', 'AP', None), (None, '', 'Blr')]
schema="City1 string, City2 string, City3 string"
df2=spark.createDataFrame(data, schema)
df2.show()
df3=df2.withColumn("City1",when(df2.City1=="",None).otherwise(df2.City1))\
   .withColumn("City2",when(df2.City2=="",None).otherwise(df2.City2))\
   .withColumn("City3",when(df2.City3=="",None).otherwise(df2.City3))
df3.show()

df3.withColumn("Result",coalesce(df3.City1,df3.City2,df3.City3)).select('Result').show()



+-----+-----+-----+
|City1|City2|City3|
+-----+-----+-----+
|  Goa|     |   AP|
|     |   AP| NULL|
| NULL|     |  Blr|
+-----+-----+-----+

+-----+-----+-----+
|City1|City2|City3|
+-----+-----+-----+
|  Goa| NULL|   AP|
| NULL|   AP| NULL|
| NULL| NULL|  Blr|
+-----+-----+-----+

+------+
|Result|
+------+
|   Goa|
|    AP|
|   Blr|
+------+



In [43]:
data=(['Alice','Tennis,batminton'],['bob','Tennis,Cricket'],['Julie','Cricket,Carroms'])

df1=spark.createDataFrame(data,['name','hobbies'])

df=df1.select(df1.name,split(df1.hobbies,',').alias('hobbies'))
df.select(df.name,explode(df.hobbies).alias('hobbies')).show()



+-----+---------+
| name|  hobbies|
+-----+---------+
|Alice|   Tennis|
|Alice|batminton|
|  bob|   Tennis|
|  bob|  Cricket|
|Julie|  Cricket|
|Julie|  Carroms|
+-----+---------+



In [45]:
#Alternate Approach (Can be ignored)
data=[('Goa', '', 'AP'), ('', 'AP', None), (None, '', 'Blr')]
schema="City1 string, City2 string, City3 string"
df2=spark.createDataFrame(data, schema)

df2.withColumn("Result",coalesce(when(df2.City1=="",None).otherwise(df2.City1),(when(df2.City2=="",None).otherwise(df2.City2)),when(df2.City3=="",None).otherwise(df2.City3))).select('Result').show()

+------+
|Result|
+------+
|   Goa|
|    AP|
|   Blr|
+------+



In [60]:
data=[[1,'Steve','Maths',90],[1,'Steve','Science',100],[2,'Philips','Maths',90],[2,'Philips','Science',90]]

df=spark.createDataFrame(data,['ID','Name','Subject','Marks'])
df.show()
d=df.groupBy('ID','Name').agg(((sum(df.Marks))/count('*')).alias('Average'))

d.withColumn("Grade",when(d.Average>=90,'Distinction').when(d.Average>80,'First Class').otherwise('Fail')).show()

+---+-------+-------+-----+
| ID|   Name|Subject|Marks|
+---+-------+-------+-----+
|  1|  Steve|  Maths|   90|
|  1|  Steve|Science|  100|
|  2|Philips|  Maths|   90|
|  2|Philips|Science|   90|
+---+-------+-------+-----+

+---+-------+-------+-----------+
| ID|   Name|Average|      Grade|
+---+-------+-------+-----------+
|  1|  Steve|   95.0|Distinction|
|  2|Philips|   90.0|Distinction|
+---+-------+-------+-----------+



In [73]:
from pyspark.sql.functions import *

from pyspark.sql.window import Window
data=[[1,'Sudhamsh','Data',80000],[2,'Tanvi','Automation',81000],[3,'Varma','Data',75000],[4,'Nibbi','Automation',82000]]

df=spark.createDataFrame(data,['ID','Name','Department','Salary'])
w=Window.partitionBy(df.Department).orderBy(df['Salary'].desc)
df.withColumn('Rank',rank().over(w)).show()

PySparkTypeError: [NOT_COLUMN_OR_STR] Argument `col` should be a Column or str, got method.

In [86]:
sales_data = [
('Product_A', 100, 150, 200),
('Product_B', 120, 80, 110),
('Product_C', 90, 130, 180),
('Product_D', 200, 160, 120)
]


df = spark.createDataFrame(sales_data,['Product','Jan_Sales','Feb_Sales','Mar_Sales'])
df.show()
updf=df.unpivot('Product',['Jan_Sales','Feb_sales','Mar_sales'],'Month','Sales')
updf.show()





+---------+---------+---------+---------+
|  Product|Jan_Sales|Feb_Sales|Mar_Sales|
+---------+---------+---------+---------+
|Product_A|      100|      150|      200|
|Product_B|      120|       80|      110|
|Product_C|       90|      130|      180|
|Product_D|      200|      160|      120|
+---------+---------+---------+---------+

+---------+---------+-----+
|  Product|    Month|Sales|
+---------+---------+-----+
|Product_A|Jan_Sales|  100|
|Product_A|Feb_sales|  150|
|Product_A|Mar_sales|  200|
|Product_B|Jan_Sales|  120|
|Product_B|Feb_sales|   80|
|Product_B|Mar_sales|  110|
|Product_C|Jan_Sales|   90|
|Product_C|Feb_sales|  130|
|Product_C|Mar_sales|  180|
|Product_D|Jan_Sales|  200|
|Product_D|Feb_sales|  160|
|Product_D|Mar_sales|  120|
+---------+---------+-----+



AttributeError: 'DataFrame' object has no attribute 'pivot'